In [3]:
import os
import pandas as pd

In [ ]:
folder_path = './output/'  # Use current directory
script_name = 'preprocessing.ipynb'  # Name of the script to exclude

merged_data = pd.DataFrame()

for filename in os.listdir(folder_path):
    if (
        filename.startswith('articles_data') and
        'articles_links' not in filename and
        filename != script_name
    ):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        merged_data = pd.concat([merged_data, df], ignore_index=True)

merged_data.to_csv(os.path.join(folder_path, 'merged_articles_data.csv'), index=False) 

In [2]:
df = pd.read_csv('./output/merged_articles_data.csv')
df.shape


(1845611, 3)

In [3]:
df.head()

,url,title,abstract
0,https://link.springer.com/article/10.1007/s131...,Sub-Category Generalism About Conspiracy Theories,I argue for a kind of sub-category generalism ...
1,https://link.springer.com/article/10.1007/s131...,Framing Effects in Object Perception,In this paper we argue that object perception ...
2,https://link.springer.com/article/10.1007/s131...,Desire and Motivation in Predictive Processing...,The predictive processing theory refers to a f...
3,https://link.springer.com/article/10.1007/s131...,"Consciousness, Causation, and Confusion","In recent decades, various sources of scientif..."
4,https://link.springer.com/article/10.1007/s131...,Cognitive Behavioural Virtue – how to Acquire ...,The application and practice of virtue ethics ...


In [4]:
df = df.drop(columns=['introduction', 'conclusion'])


KeyError: "['introduction', 'conclusion'] not found in axis"

In [5]:
df = df.drop_duplicates(subset='url')

In [6]:
df.shape

(1845312, 3)

In [7]:
df = df.dropna(subset=['abstract'])
df.shape

(1843756, 3)

In [8]:
df.head()

,url,title,abstract
0,https://link.springer.com/article/10.1007/s131...,Sub-Category Generalism About Conspiracy Theories,I argue for a kind of sub-category generalism ...
1,https://link.springer.com/article/10.1007/s131...,Framing Effects in Object Perception,In this paper we argue that object perception ...
2,https://link.springer.com/article/10.1007/s131...,Desire and Motivation in Predictive Processing...,The predictive processing theory refers to a f...
3,https://link.springer.com/article/10.1007/s131...,"Consciousness, Causation, and Confusion","In recent decades, various sources of scientif..."
4,https://link.springer.com/article/10.1007/s131...,Cognitive Behavioural Virtue – how to Acquire ...,The application and practice of virtue ethics ...


In [9]:
# Remove rows with missing values in the 'title' column
df = df.dropna(subset=['title'])

# Remove leading and trailing whitespace from 'title' and 'abstract' columns
df['title'] = df['title'].str.strip()
df['abstract'] = df['abstract'].str.strip()

# Display the first few rows of the preprocessed dataframe
df.shape

(1843756, 3)

In [ ]:
df.to_csv('cleaned_data.csv', index=False)

In [4]:
import re
from bs4 import BeautifulSoup

df = pd.read_csv('cleaned_data.csv')

# Function to clean text
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# Apply the clean_text function to 'title' and 'abstract' columns
df['title'] = df['title'].apply(clean_text)
df['abstract'] = df['abstract'].apply(clean_text)

# Display the first few rows of the cleaned dataframe
df.head()

/tmp/ipykernel_9476/3653211283.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


: 

In [1]:
import os
print(f"File Size: {os.path.getsize('cleaned_data.csv') / (1024 * 1024):.2f} MB")


File Size: 2026.04 MB


In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# Function to clean text safely
def clean_text(text):
    if pd.isna(text):  # Handle NaNs
        return ""
    text = BeautifulSoup(str(text), "html.parser").get_text()
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Process the file in chunks
chunk_size = 10000  # Load 10,000 rows at a time
cleaned_data = []

for chunk in pd.read_csv("cleaned_data.csv", chunksize=chunk_size):
    chunk["title"] = chunk["title"].apply(clean_text)
    chunk["abstract"] = chunk["abstract"].apply(clean_text)
    cleaned_data.append(chunk)  # Store processed chunks

# Combine all chunks into a single DataFrame
df = pd.concat(cleaned_data, ignore_index=True)
df.head()

/tmp/ipykernel_10943/1360384360.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(str(text), "html.parser").get_text()
/tmp/ipykernel_10943/1360384360.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(str(text), "html.parser").get_text()
/tmp/ipykernel_10943/1360384360.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(str(text), "html.parser").get_text()
/tmp/ipykernel_10943/1360384360.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(str(text), "html.

In [25]:
df.to_csv('preprocessed_data.csv', index=False)